In [170]:
from nltk.corpus import twitter_samples

In [171]:
# loading data

positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')
neutral = twitter_samples.strings('tweets.20150430-223406.json')

tweet_tokens = twitter_samples.tokenized('positive_tweets.json')

In [172]:
# Lemmatization

from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer

def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

In [173]:
# Cleaning

import re, string

def remove_noise(tweet_tokens, stop_words = ()):

    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [174]:
# Stop words and tokenization

from nltk.corpus import stopwords
stop_words = stopwords.words('english')

print(remove_noise(tweet_tokens[0], stop_words))

positive_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
negative_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')
neutral_tweet_tokens = twitter_samples.tokenized('tweets.20150430-223406.json')

positive_cleaned_tokens_list = []
negative_cleaned_tokens_list = []
neutral_cleaned_tokens_list = []

for tokens in positive_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in negative_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))
    
for tokens in neutral_tweet_tokens:
    neutral_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

['#followfriday', 'top', 'engage', 'member', 'community', 'week', ':)']


In [175]:
from nltk import FreqDist

freq_dist_pos = FreqDist(all_pos_words)
print(freq_dist_pos.most_common(10))

[]


In [176]:
# Extracting tokens

def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)
neutral_tokens_for_model = get_tweets_for_model(neutral_cleaned_tokens_list)

In [177]:
# test train partition

import random

positive_dataset = [(tweet_dict, "Positive")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Negative")
                     for tweet_dict in negative_tokens_for_model]

neutral_dataset = [(tweet_dict, "Neutral")
                     for tweet_dict in neutral_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:8000]
test_data = dataset[8000:]

In [178]:
dataset

[({':(': True,
   'betis': True,
   'close': True,
   'greatings': True,
   'hello': True,
   'info': True,
   'interest': True,
   'join': True,
   'jonathas': True,
   'possible': True},
  'Negative'),
 ({':(': True,
   'anything': True,
   "can't": True,
   'even': True,
   'pay': True,
   'play': True,
   'properly': True,
   'solution': True,
   'website': True},
  'Negative'),
 ({':(': True,
   'aigooo': True,
   'cant': True,
   'get': True,
   'riri': True,
   'seungchan': True},
  'Negative'),
 ({'...': True, ':(': True, 'yeah': True}, 'Negative'),
 ({'2ne1': True, ':d': True, 'beautiful': True, 'day': True, 'good': True},
  'Positive'),
 ({'...': True,
   ':)': True,
   'attack': True,
   'attentive': True,
   'criticise': True,
   'every': True,
   'gesture': True,
   'little': True,
   'must': True,
   'notice': True,
   'one': True,
   'pedal': True,
   'stroke': True,
   'thing': True,
   'word': True,
   '´': True},
  'Positive'),
 ({':-(': True,
   'ask': True,
   'bos'

In [179]:
# NaiveBayes Classifier model

from nltk import classify
from nltk import NaiveBayesClassifier
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))

Accuracy: 0.996
Most Informative Features
                      :( = True           Negati : Positi =   2367.2 : 1.0
                      :) = True           Positi : Negati =   1132.2 : 1.0
                follower = True           Positi : Negati =     24.5 : 1.0
                     bam = True           Positi : Negati =     23.5 : 1.0
                     sad = True           Negati : Positi =     21.6 : 1.0
                  arrive = True           Positi : Negati =     21.3 : 1.0
                     x15 = True           Negati : Positi =     18.4 : 1.0
                      aw = True           Negati : Positi =     17.8 : 1.0
               community = True           Positi : Negati =     17.6 : 1.0
              appreciate = True           Positi : Negati =     15.6 : 1.0
None


In [180]:
# Saving model

import pickle

filename = 'finalized_model.sav'
pickle.dump(classifier, open(filename, 'wb'))